<h1>Connect to Database<h1>

In [9]:
from sqlalchemy import create_engine
import pandas as pd

In [10]:
from dotenv import load_dotenv
import os

load_dotenv('pwd.env')

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv("DB_HOST")

In [11]:
username = db_user
password = db_password
host = db_host
port = '5433'
database = 'House_Power_DWH'

connection_string = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

try:
    with engine.connect() as connection:
        print("connection was successful!")
except Exception as e:
    print("connectionfailed:",e)

connection was successful!


<h1>Load Files To DWH<h1>

In [16]:

cleaned_power =pd.read_csv(r'D:\My Projects\grad project.DEPI\HOUSE_POWER_PROJECT\HOUSE_POWER_PROJECT\Data set\cleaned_power.csv', sep = ',')
cleaned_power.to_sql('cleaned_power', engine, if_exists='replace', index = False, schema='public')

356

In [17]:
cleaned_sub_meterings =pd.read_csv(r'D:\My Projects\grad project.DEPI\HOUSE_POWER_PROJECT\HOUSE_POWER_PROJECT\Data set\cleaned_sub_meterings.csv', sep = ',')
cleaned_sub_meterings .to_sql('cleaned_sub_meterings ', engine, if_exists='replace', index = False, schema='public')

356

In [18]:
cleaned_voltage_current =pd.read_csv(r'D:\My Projects\grad project.DEPI\HOUSE_POWER_PROJECT\HOUSE_POWER_PROJECT\Data set\cleaned_voltage_current.csv', sep = ',')
cleaned_voltage_current .to_sql('cleaned_voltage_current  ', engine, if_exists='replace', index = False, schema='public')

356

<h1>Create Fact Table<h1>

In [22]:
from sqlalchemy import text
with engine.connect() as connection:
    connection.execute(text('''
    CREATE TABLE fact_energy_usage AS
    SELECT
        cp."Index",
        cp."Global_active_power KW",
        cp."Global_reactive_power KW",
        cp."Apparent_power_KVA",
        csm."Sub_metering_1 J" ,
        csm."Sub_metering_2 J" ,
        csm."Sub_metering_3 J",
        cvc."Voltage V", 
        cvc."Global_intensity A" ,
        cvc."Date",
        cvc."Time"
    FROM
        "public"."cleaned_power" cp
    JOIN
        "public"."cleaned_sub_meterings " csm ON cp."Index" = csm."Index"
    JOIN
        "public"."cleaned_voltage_current  " cvc ON cp."Index" = cvc."Index";
    '''))